In [ ]:
%env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
# from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
from fgz.training.xirl_trainer import XIRLTrainer
from xirl_config import XIRLConfig
import torch
import wandb
from tqdm import tqdm
from fgz.data_utils.generate_xirl_targets import generate_target

In [ ]:
torch.cuda.empty_cache()  # fix memory leaks

In [ ]:
config = XIRLConfig(
    enabled_tasks=[2],  # just cave 
    disable_fmc_detection=True,  # if true, only classification will occur. 
    use_wandb=False,
    data_workers=1,
    embed_batch_size=4,
    batch_size=4,
    num_frames_per_pair=16,
    num_frames_per_trajectory_to_load=32,
)

In [ ]:
# minerl_env = gym.make('MineRLBasaltMakeWaterfall-v0')
# agent = get_agent(config)
# dynamics_env = get_dynamics_environment(config)

In [ ]:
# agent.device

In [ ]:
# dynamics_function_optimizer = torch.optim.Adam(
#     dynamics_env.dynamics_function.parameters(),
#     lr=0.000005,
#     weight_decay=1e-4,
# )

In [ ]:
dataset_path = config.dataset_paths[0]
print(dataset_path)
trainer = XIRLTrainer(config)

In [ ]:
target_embedding = generate_target(config, trainer.model, dataset_path)

In [ ]:
target_embedding

In [ ]:
from fgz.data_utils.data_handler import ContiguousTrajectoryDataLoader
from fgz.data_utils.visualize_xirl import plot_xirl_reward_over_time

loader = ContiguousTrajectoryDataLoader(dataset_path, max_num_trajectories=3)
trajectory = loader.sample()

In [ ]:
trajectory, len(trajectory)

In [ ]:
rewards = plot_xirl_reward_over_time(config, trajectory, trainer.model, target_embedding)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(rewards.numpy())

In [ ]:
# import ray
# ray.get(trainer.train_on_pair())

for _ in range(2):
    trainer.train_on_pair()

In [ ]:
trainer.get_nbytes_stored()